In [1]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from openai import OpenAI

In [3]:
requests.get("http://localhost:11434").content

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

model_name="llama3.2"

In [5]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
#from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationBufferMemory

In [7]:
# Initialize the Ollama model
llm = ChatOllama(
    model=model_name,   # change to mistral / qwen2.5 if you want
    temperature=0
)

## Generating Multiple Reasoning Paths

In [14]:
def generate_multiple_paths(problem, num_paths=3):
    """
    Generate multiple reasoning paths for a given problem.
    
    Args:
    problem (str): The problem statement.
    num_paths (int): Number of reasoning paths to generate.
    
    Returns:
    list: A list of generated reasoning paths.
    """
    prompt_template = PromptTemplate(
        input_variables=["problem", "path_number"],
        template="""Solve the following problem using a unique approach. This is reasoning path {path_number}.
        Problem: {problem}
        Reasoning path {path_number}:"""
    )

    paths = []
    for i in range(num_paths):
        chain = prompt_template | llm
        response = chain.invoke({"problem": problem, "path_number": i+1}).content
        paths.append(response)
    
    return paths

In [16]:
problem = "A ball is thrown upwards with an initial velocity of 20 m/s. How high will it go?"
paths = generate_multiple_paths(problem)

for i, path in enumerate(paths, 1):
    print(f"Path {i}:\n{path}\n")

Path 1:
This problem can be solved by considering the motion of the ball as a projectile under the influence of gravity. Since the ball is thrown upwards, its vertical component of velocity decreases due to gravity until it reaches its maximum height.

Let's consider the following steps:

1. The initial upward velocity of 20 m/s will decrease at a rate of 9.8 m/s^2 (acceleration due to gravity) until it becomes zero.
2. At this point, the ball has reached its maximum height and momentarily comes to rest.
3. From this point onwards, the ball starts falling downwards under the influence of gravity.

To find the maximum height, we can use the following equation:

v^2 = u^2 + 2as

where:
- v is the final velocity (0 m/s at the maximum height)
- u is the initial velocity (20 m/s upwards)
- a is the acceleration due to gravity (-9.8 m/s^2, negative because it's acting downwards)
- s is the displacement (height) we want to find

Rearranging the equation to solve for s, we get:

s = v^2 / 2a
=

## Aggregating Results

In [21]:
def aggregate_results(paths):
    """
    Aggregate results from multiple reasoning paths.
    
    Args:
    paths (list): List of reasoning paths.
    
    Returns:
    str: The most consistent answer.
    """
    prompt_template = PromptTemplate(
        input_variables=["paths"],
        template="""Analyze the following reasoning paths and determine the most consistent answer. If there are discrepancies,
        explain why and provide the most likely correct answer.
        Reasoning paths:
        {paths}
        
        Most consistent answer:"""
    )

    chain = prompt_template | llm
    response = chain.invoke({"paths": "\n".join(paths)}).content
    return response

In [23]:
aggregated_result = aggregate_results(paths)
print("Aggregated Result:\n", aggregated_result)

Aggregated Result:
 The most consistent answer is:

s(t) = s0 + v0t - (1/2)gt^2
= 0 + 20(1.02) - (1/2)(-9.8)(1.02)^2
≈ 20.4 meters

This approach allows us to model the motion of the ball as a continuous process, which can be more intuitive and easier to work with than discrete step-by-step calculations.

The other approaches have discrepancies:

* The first approach assumes that the ball comes to rest at its maximum height and then starts falling downwards, which is not accurate.
* The second approach uses the equation v(t) = v0 - gt to find when the velocity becomes zero, but it does not take into account the initial upward velocity correctly.

The third approach is more consistent because it models the motion of the ball as a continuous process and takes into account the initial upward velocity correctly. It also provides a clear and intuitive solution for finding the maximum height above the ground.


## Self-Consistency Check

In [26]:
def self_consistency_check(problem, aggregated_result):
    """
    Perform a self-consistency check on the aggregated result.
    
    Args:
    problem (str): The original problem statement.
    aggregated_result (str): The aggregated result to check.
    
    Returns:
    str: An evaluation of the result's consistency and reliability.
    """
    prompt_template = PromptTemplate(
        input_variables=["problem", "result"],
        template="""Evaluate the consistency and reliability of the following result for the given problem.
        Problem: {problem}
        Result: {result}
        
        Evaluation (consider factors like logical consistency, adherence to known facts, and potential biases):"""
    )

    chain = prompt_template | llm
    response = chain.invoke({"problem": problem, "result": aggregated_result}).content
    return response

In [28]:
consistency_evaluation = self_consistency_check(problem, aggregated_result)
print("Self-Consistency Evaluation:\n", consistency_evaluation)

Self-Consistency Evaluation:
 The evaluation of the result's consistency and reliability is as follows:

1.  **Logical Consistency:** The approach used in the result is based on the equation for uniformly accelerated motion under gravity, which is a fundamental concept in physics. The use of this equation to model the ball's motion as a continuous process is logical and consistent with established principles.

2.  **Adherence to Known Facts:** The result adheres to known facts about projectile motion under gravity. It correctly accounts for the initial upward velocity, acceleration due to gravity, and the resulting maximum height reached by the ball.

3.  **Potential Biases:** There are no apparent biases in the approach used in the result. However, it's worth noting that the assumption of a flat, horizontal surface might not be accurate in all scenarios (e.g., on an inclined plane or in a real-world environment with air resistance). Nonetheless, for simplicity and to maintain consiste

## Applying different Problem type

In [31]:
def solve_problem(problem):
    """
    Solve a problem using multiple reasoning paths, aggregation, and self-consistency check.
    
    Args:
    problem (str): The problem statement.
    
    Returns:
    tuple: (aggregated_result, consistency_evaluation)
    """
    paths = generate_multiple_paths(problem)
    aggregated_result = aggregate_results(paths)
    consistency_evaluation = self_consistency_check(problem, aggregated_result)
    return aggregated_result, consistency_evaluation

# Example problems
problems = [
    "What is the capital of France?",
    "Explain the concept of supply and demand in economics.",
    "If a train travels at 60 km/h, how long will it take to cover 180 km?"
]

for problem in problems:
    print(f"Problem: {problem}")
    result, evaluation = solve_problem(problem)
    print("Aggregated Result:\n", result)
    print("\nConsistency Evaluation:\n", evaluation)
    print("\n" + "-"*50 + "\n")

Problem: What is the capital of France?
Aggregated Result:
 After analyzing all three reasoning paths, I can conclude that the most consistent answer is:

**Paris is likely to be the capital of France.**

Here's why:

1. **Consistency across paths**: All three paths lead to the conclusion that Paris is a significant urban center in France, culturally influential, and historically important.
2. **Lack of contradictory evidence**: None of the reasoning paths present any contradictory information that would suggest another city in France is the capital.
3. **Use of observation and pattern recognition**: Each path uses observation and pattern recognition to gather information about the concept of "capital" and its relationship with France, which is a consistent approach.

The only discrepancy between the three paths is the level of detail and specificity provided. Path 1 provides more general observations, while Paths 2 and 3 offer more specific examples and patterns. However, all three pa

In [33]:
# Function to display model outputs
def display_output(output):
    """Display the model's output in a formatted manner."""
    print("Model Output:")
    print("-" * 40)
    print(output)
    print("-" * 40)
    print()

## Setting Constraints for Model Outputs

In [36]:
constrained_prompt = PromptTemplate(
    input_variables=["product", "target_audience", "tone", "word_limit"],
    template="""Create a product description for {product} targeted at {target_audience}.
    Use a {tone} tone and keep it under {word_limit} words.
    The description should include:
    1. A catchy headline
    2. Three key features
    3. A call to action
    
    Product Description:
    """
)

# Generate the constrained output
input_variables = {
    "product": "smart water bottle",
    "target_audience": "health-conscious millennials",
    "tone": "casual and friendly",
    "word_limit": "75"
}

chain = constrained_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
**Hydrate Your Way to Happiness**

Stay on top of your game with our smart water bottle! 

Track your daily hydration goals with our built-in sensor
Monitor your water intake and receive reminders to drink up
Get access to personalized nutrition advice and healthy recipes via our mobile app

Download the app now and start sipping your way to a healthier you!
----------------------------------------



## Implementing Rule-based generation

In [39]:
job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement
    
    Format the output as follows:
    COMPANY: [Company Description]
    
    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]
    
    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]
    
    EEO: [Equal Opportunity Statement]
    """
)

# Generate the rule-based output
input_variables = {
    "job_title": "Senior Software Engineer",
    "company": "TechInnovate Solutions",
    "location": "San Francisco, CA",
    "experience": "5+"
}

chain = job_posting_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
COMPANY: TechInnovate Solutions is a leading technology firm in San Francisco, CA, dedicated to delivering innovative software solutions to clients across various industries. With a strong focus on collaboration and customer satisfaction, we strive to be at the forefront of technological advancements.

RESPONSIBILITIES:
- Design and develop complex software systems that meet client requirements
- Collaborate with cross-functional teams to identify and prioritize project needs
- Implement automated testing and deployment scripts to ensure seamless system integration
- Develop and maintain technical documentation to facilitate knowledge sharing within the team
- Lead code reviews and mentor junior engineers to promote best practices

QUALIFICATIONS:
- Possess at least 5+ years of experience in software engineering, with a strong track record of delivering high-quality solutions
- Hold a Bachelor's or Master's degree in Computer Scien

## Structured Outputs

In [56]:
from pydantic import BaseModel

In [58]:
class JobPosting(BaseModel):
    company_description: str
    responsibilities: list[str]
    qualifications: list[str]
    eeo_statement: str



structured_llm = llm.with_structured_output(JobPosting)


prompt = ChatPromptTemplate.from_template("""
Create a job posting for a {job_title} position at {company} in {location}.
The candidate should have {experience} years of experience.

Follow these rules:
1. Company description → 2 sentences
2. 5 key responsibilities → action verbs
3. 5 qualifications → single sentence each
4. Equal opportunity statement at the end
""")



chain = prompt | structured_llm


input_data = {
    "job_title": "AI Engineer",
    "company": "TechNova",
    "location": "Bangalore",
    "experience": "3+"
}



result = chain.invoke(input_data)


print("\nPARSED OUTPUT:\n")

print("COMPANY DESCRIPTION:")
print(result.company_description, "\n")

print("RESPONSIBILITIES:")
for r in result.responsibilities:
    print("-", r)
print()

print("QUALIFICATIONS:")
for q in result.qualifications:
    print("-", q)
print()

print("EEO STATEMENT:")
print(result.eeo_statement)


PARSED OUTPUT:

COMPANY DESCRIPTION:
TechNova is a leading technology company that specializes in developing innovative AI and machine learning solutions for various industries. With a strong focus on research and development, we are committed to pushing the boundaries of what is possible with AI. 

RESPONSIBILITIES:
- Design and develop intelligent systems using cutting-edge AI technologies
- Collaborate with cross-functional teams to integrate AI models into existing applications
- Develop and deploy machine learning algorithms for predictive analytics and data science projects
- Conduct research and stay up-to-date with the latest advancements in AI and machine learning
- Work closely with stakeholders to understand business requirements and deliver tailored AI solutions

QUALIFICATIONS:
- Bachelor's or Master's degree in Computer Science, Electrical Engineering, or related field
- 3+ years of experience in AI engineering, software development, or a related field
- Strong proficien

In [61]:
review_prompt = PromptTemplate(
    input_variables=["product", "rating", "pros", "cons", "word_limit"],
    template="""Write a product review for {product} with the following constraints:

1. The review should have a {rating}-star rating (out of 5)
2. Include exactly {pros} pros and {cons} cons
3. Use between 2 and 3 sentences for each pro and con
4. The entire review should be under {word_limit} words
5. End with a one-sentence recommendation

Format the review as follows:

Rating: [X] out of 5 stars

Pros:
1. [Pro 1]
2. [Pro 2]

Cons:
1. [Con 1]
2. [Con 2]

Recommendation: [One-sentence recommendation]
"""
)



input_data = {
    "product": "Smartphone X",
    "rating": "4",
    "pros": "3",
    "cons": "2",
    "word_limit": "200"
}


chain = review_prompt | llm



response = chain.invoke(input_data)



print(response.content)

Rating: 4 out of 5 stars

Pros:
1. The Smartphone X has an impressive camera system, capable of capturing high-quality photos and videos even in low-light conditions. The advanced features such as portrait mode and bokeh effect are also a great addition to the camera app. Overall, the camera is one of the standout features of this device.
2. The phone's battery life is excellent, lasting up to 2 days with moderate usage. This is thanks to the efficient processor and power-saving features that help reduce battery drain. Whether you're using it for gaming or social media, the battery will keep going all day long.
3. The Smartphone X has a sleek and durable design, making it perfect for those who want a premium feel without breaking the bank. The phone's weight is also well-balanced, making it comfortable to hold and use.

Cons:
1. The phone's storage capacity could be better, with only 128GB available. This may not be enough for users who need more space for their files and apps.
2. The 

# Role Prompting

In [64]:
tech_writer_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""You are a technical writer specializing in creating clear and concise documentation for software products.
    Your task is to write a brief explanation of {topic} for a user manual.
    Please provide a 2-3 sentence explanation that is easy for non-technical users to understand."""
)

chain = tech_writer_prompt | llm
response = chain.invoke({"topic": "cloud computing"})
print(response.content)

Here's a possible explanation:

"Cloud computing is a way to store, manage, and access data and applications over the internet, rather than on your own computer or device. This means you can use software and access files from anywhere with an internet connection, without having to worry about storing them on your local hard drive. Think of it like renting a virtual storage space that's accessible from anywhere, at any time."


In [66]:
financial_advisor_prompt = PromptTemplate(
    input_variables=["client_situation"],
    template="""You are a seasoned financial advisor with over 20 years of experience in personal finance, investment strategies, and retirement planning.
    You have a track record of helping clients from diverse backgrounds achieve their financial goals.
    Your approach is characterized by:
    1. Thorough analysis of each client's unique financial situation
    2. Clear and jargon-free communication of complex financial concepts
    3. Ethical considerations in all recommendations
    4. A focus on long-term financial health and stability

    Given the following client situation, provide a brief (3-4 sentences) financial advice:
    {client_situation}

    Your response should reflect your expertise and adhere to your characteristic approach."""
)

chain = financial_advisor_prompt | llm
response = chain.invoke({"client_situation": "A 35-year-old professional earning $80,000 annually, with $30,000 in savings, no debt, and no retirement plan."})
print(response.content)

Based on the client's situation, I would recommend that they prioritize building an emergency fund to cover 3-6 months of living expenses, which is currently lacking. Additionally, I would suggest exploring tax-advantaged retirement accounts such as a 401(k) or IRA, and contributing at least 10% of their income towards it. This will help them establish a solid foundation for long-term financial security and take advantage of compound interest to grow their savings over time. By starting early and being consistent, they can set themselves up for a more stable financial future.


## Comparing Responsibilities in Different roles

In [69]:
roles = [
    ("Scientist", "You are a research scientist specializing in climate change. Explain the following concept in scientific terms:"),
    ("Teacher", "You are a middle school science teacher. Explain the following concept in simple terms suitable for 12-year-old students:"),
    ("Journalist", "You are a journalist writing for a popular science magazine. Explain the following concept in an engaging and informative manner for a general adult audience:")
]

topic = "The greenhouse effect"

for role, description in roles:
    role_prompt = PromptTemplate(
        input_variables=["topic"],
        template=f"{description} {{topic}}"
    )
    chain = role_prompt | llm
    response = chain.invoke({"topic": topic})
    print(f"\n{role}'s explanation:\n")
    print(response.content)
    print("-" * 50)


Scientist's explanation:

The greenhouse effect is a complex, natural process that plays a crucial role in regulating Earth's temperature and maintaining a habitable environment for life on our planet.

**Definition:** The greenhouse effect refers to the trapping of infrared radiation (heat) by certain gases present in the Earth's atmosphere. These gases, primarily carbon dioxide (CO2), methane (CH4), water vapor (H2O), and others, absorb and re-emit infrared radiation emitted by the Earth's surface, thereby warming the planet.

**Mechanism:**

1. **Solar Radiation:** The sun emits solar radiation, which enters the Earth's atmosphere.
2. **Infrared Emission:** The Earth's surface absorbs solar radiation and converts it into heat energy, emitting infrared radiation (heat) back into the atmosphere.
3. **Greenhouse Gases:** Certain gases in the atmosphere, such as CO2, CH4, and H2O, absorb infrared radiation emitted by the Earth's surface.
4. **Trapping Heat:** These greenhouse gases re-